# SageMaker JumpStart - Llama instruction tuning 

This notebook demonstrates how to use the SageMaker Python SDK to fine-tune the pre-trained Llama model for your instruction tuning dataset.

## Set up

First, install necessary packages. Restart the kernel after executing the cell below for the first time.

In [ ]:
%pip install --upgrade --quiet datasets

In [ ]:
import json

import pandas as pd
from datasets import load_dataset
from IPython.display import display
from IPython.display import HTML
from sagemaker import Session
from sagemaker.jumpstart.estimator import JumpStartEstimator
from sagemaker.s3 import S3Uploader

Select your desired model ID. You can search for available models in the [Built-in Algorithms with pre-trained Model Table](https://sagemaker.readthedocs.io/en/stable/doc_utils/pretrainedmodels.html).

In [ ]:
model_id = "meta-textgeneration-llama-3-2-1b"

## Dataset preparation

You can fine-tune on the dataset with domain adaptation format or instruction tuning format. In this demo, we will use a subset of [Dolly dataset](https://huggingface.co/datasets/databricks/databricks-dolly-15k) in an instruction tuning format. The Dolly dataset contains roughly 15,000 instruction following records for various categories such as question answering, summarization, information extraction etc. It is available under Apache 2.0 license. We will select the summarization examples for fine-tuning.

Training data is formatted in JSON lines (.jsonl) format, where each line is a dictionary representing a single data sample. All training data must be in a single folder, however it can be saved in multiple jsonl files. The training folder can also contain a template.json file describing the input and output formats.

To train your model on a collection of unstructured dataset (text files), please see the section [Example fine-tuning with Domain-Adaptation dataset format](#Example-fine-tuning-with-Domain-Adaptation-dataset-format) in the Appendix.


In [ ]:
dolly_dataset = load_dataset("databricks/databricks-dolly-15k", split="train")

To train for question answering/information extraction, you can replace the filter in next line to `example["category"] == "closed_qa"/"information_extraction"`.

In [ ]:
summarization_dataset = dolly_dataset.filter(lambda example: example["category"] == "summarization")
summarization_dataset = summarization_dataset.remove_columns("category")

Now, split the dataset into train and test sets, where test data is used at the end of this notebook.

In [ ]:
train_and_test_dataset = summarization_dataset.train_test_split(test_size=0.1)

Next, create a prompt template for using the data in an instruction format for the training job. You will also use this template during model inference.

In [ ]:
template = {
    "prompt": (
        "Below is an instruction that describes a task, paired with an "
        "input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{context}\n\n"
    ),
    "completion": " {response}",
}

Save the training data and template to local files to be used for training.

In [ ]:
train_and_test_dataset["train"].to_json("train.jsonl")

with open("template.json", "w") as f:
    json.dump(template, f)

Finally, upload the prepared dataset to S3 for use in fine-tuning.


In [ ]:
session = Session()
output_bucket = session.default_bucket()
default_bucket_prefix = session.default_bucket_prefix

# If a default bucket prefix is specified, prepend it to the output bucket
if default_bucket_prefix:
    default_path = f"{output_bucket}/{default_bucket_prefix}"
else:
    default_path = output_bucket

local_data_file = "train.jsonl"
train_data_location = f"s3://{default_path}/dolly_dataset"
S3Uploader.upload(local_data_file, train_data_location)
S3Uploader.upload("template.json", train_data_location)
print(f"Training data: {train_data_location}")

## Train the model

In this section you will fine-tune the model on the summarization dataset from Dolly. Finetuning scripts are based on scripts provided by [this repo](https://github.com/facebookresearch/llama-recipes/tree/main). To learn more about the fine-tuning scripts, please checkout section [5. Few notes about the fine-tuning method](#5.-Few-notes-about-the-fine-tuning-method). For a list of supported hyper-parameters and their default values, please see section [3. Supported Hyper-parameters for fine-tuning](#3.-Supported-Hyper-parameters-for-fine-tuning). By default, these models train via domain adaptation, so you must indicate instruction tuning through the `instruction_tuned` hyperparameter.


In [ ]:
# set custom Hyperparameters for fine-tuning the model
hyperparameters = {
    "instruction_tuned": "True",
    "epoch": "5",
    "max_input_length": "1024",
    "chat_dataset": "False",
}

# pass in None to use the default training_instance_type
training_instance_type = None

# attach a custom session to the Estimator class below
sagemaker_session = session

# accept the eula by passing in Environment values to the Estimator class below
environment = {"accept_eula": "false"}  # set "accept_eula": "true" to accept the EULA for gated models

# optionally, disable output compression by passing to the Estimator class below
disable_output_compression = True

estimator = JumpStartEstimator(
    model_id=model_id,
    hyperparameters=hyperparameters,
    instance_type=training_instance_type
)

In [ ]:
estimator.fit({"training": train_data_location})

If your studio kernel dies and you lose reference to the estimator object, please see section [6. Studio Kernel Dead/Creating JumpStart Model from the training Job](#6.-Studio-Kernel-Dead/Creating-JumpStart-Model-from-the-training-Job) on how to deploy endpoint using the training job name and the model id. 


## Deploy and invoke the fine-tuned model

You can deploy the fine-tuned model to an endpoint directly from the estimator.


In [ ]:
predictor = estimator.deploy()

Next, we use the test data to invoke the fine-tuned model. To do this, create a helper function to template an example datapoint and query the model. For instruction fine-tuning, we insert a special demarkation key between the example input and output text, so this is added here in the templating process for inference.

In [ ]:
def template_and_predict(example, input_output_demarkation_key="\n\n### Response:\n"):
    prompt = template["prompt"]
    inputs = prompt.format(instruction=example["instruction"], context=example["context"])
    inputs += input_output_demarkation_key
    payload = {"inputs": inputs, "parameters": {"max_new_tokens": 100}}
    response = predictor.predict(payload)
    return inputs, example["response"], response

Now iterate over a few test dataset examples and call the helper function. We aggregate these predictions into a pandas DataFrame for viewing.

In [ ]:
df = pd.DataFrame(
    [template_and_predict(x) for x in train_and_test_dataset["test"].select(range(5))],
    columns=["Inputs", "Ground Truth", "Response from fine-tuned model"],
)
display(HTML(df.to_html()))

## Clean up

In [ ]:
predictor.delete_predictor()

## Appendix

### 1. Supported Inference Parameters

This model supports the following payload parameters. You may specify any subset of these parameters when invoking an endpoint.

* **do_sample:** If True, activates logits sampling. If specified, it must be boolean.
* **max_new_tokens:** Maximum number of generated tokens. If specified, it must be a positive integer.
* **repetition_penalty:** A penalty for repetitive generated text. 1.0 means no penalty.
* **return_full_text:** If True, input text will be part of the output generated text. If specified, it must be boolean. The default value for it is False.
* **stop**: If specified, it must a list of strings. Text generation stops if any one of the specified strings is generated.
* **seed**: Random sampling seed.
* **temperature:** Controls the randomness in the output. Higher temperature results in output sequence with low-probability words and lower temperature results in output sequence with high-probability words. If `temperature` -> 0, it results in greedy decoding. If specified, it must be a positive float.
* **top_k:** In each step of text generation, sample from only the `top_k` most likely words. If specified, it must be a positive integer.
* **top_p:** In each step of text generation, sample from the smallest possible set of words with cumulative probability `top_p`. If specified, it must be a float between 0 and 1.
* **truncate:** Truncate inputs tokens to the given size.
* **typical_p:** Typical decoding mass, according to [Typical Decoding for Natural Language Generation](https://arxiv.org/abs/2202.00666).
* **best_of:** Generate best_of sequences and return the one if the highest token logprobs.
* **watermark:** Whether to perform watermarking with [A Watermark for Large Language Models](https://arxiv.org/abs/2301.10226).
* **details:** Return generation details, to include output token logprobs and IDs.
* **decoder_input_details:** Return decoder input token logprobs and IDs.
* **top_n_tokens:** Return the N most likely tokens at each step.

### 2. Dataset formatting instruction for training

We currently offer two types of fine-tuning: instruction fine-tuning and domain adaption fine-tuning. You can easily switch to one of the training 
methods by specifying parameter `instruction_tuned` being 'True' or 'False'.


#### 2.1. Domain adaptation fine-tuning
The Text Generation model can also be fine-tuned on any domain specific dataset. After being fine-tuned on the domain specific dataset, the model
is expected to generate domain specific text and solve various NLP tasks in that specific domain with **few shot prompting**.

Below are the instructions for how the training data should be formatted for input to the model.

- **Input:** A train and an optional validation directory. Each directory contains a CSV/JSON/TXT file. 
  - For CSV/JSON files, the train or validation data is used from the column called 'text' or the first column if no column called 'text' is found.
  - The number of files under train and validation (if provided) should equal to one, respectively. 
- **Output:** A trained model that can be deployed for inference. 

Below is an example of a TXT file for fine-tuning the Text Generation model. The TXT file is SEC filings of Amazon from year 2021 to 2022.

```Note About Forward-Looking Statements
This report includes estimates, projections, statements relating to our
business plans, objectives, and expected operating results that are “forward-
looking statements” within the meaning of the Private Securities Litigation
Reform Act of 1995, Section 27A of the Securities Act of 1933, and Section 21E
of the Securities Exchange Act of 1934. Forward-looking statements may appear
throughout this report, including the following sections: “Business” (Part I,
Item 1 of this Form 10-K), “Risk Factors” (Part I, Item 1A of this Form 10-K),
and “Management’s Discussion and Analysis of Financial Condition and Results
of Operations” (Part II, Item 7 of this Form 10-K). These forward-looking
statements generally are identified by the words “believe,” “project,”
“expect,” “anticipate,” “estimate,” “intend,” “strategy,” “future,”
“opportunity,” “plan,” “may,” “should,” “will,” “would,” “will be,” “will
continue,” “will likely result,” and similar expressions. Forward-looking
statements are based on current expectations and assumptions that are subject
to risks and uncertainties that may cause actual results to differ materially.
We describe risks and uncertainties that could cause actual results and events
to differ materially in “Risk Factors,” “Management’s Discussion and Analysis
of Financial Condition and Results of Operations,” and “Quantitative and
Qualitative Disclosures about Market Risk” (Part II, Item 7A of this Form
10-K). Readers are cautioned not to place undue reliance on forward-looking
statements, which speak only as of the date they are made. We undertake no
obligation to update or revise publicly any forward-looking statements,
whether because of new information, future events, or otherwise.
GENERAL
Embracing Our Future ...
```


#### 2.2. Instruction fine-tuning
The Text generation model can be instruction-tuned on any text data provided that the data 
is in the expected format. The instruction-tuned model can be further deployed for inference. 
Below are the instructions for how the training data should be formatted for input to the 
model.

Below are the instructions for how the training data should be formatted for input to the model.

- **Input:** A train and an optional validation directory. Train and validation directories should contain one or multiple JSON lines (`.jsonl`) formatted files. In particular, train directory can also contain an optional `*.json` file describing the input and output formats. 
  - The best model is selected according to the validation loss, calculated at the end of each epoch.
  If a validation set is not given, an (adjustable) percentage of the training data is
  automatically split and used for validation.
  - The training data must be formatted in a JSON lines (`.jsonl`) format, where each line is a dictionary
representing a single data sample. All training data must be in a single folder, however
it can be saved in multiple jsonl files. The `.jsonl` file extension is mandatory. The training
folder can also contain a `template.json` file describing the input and output formats. If no
template file is given, the following template will be used:
  ```json
  {
    "prompt": "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{context}",
    "completion": "{response}"
  }
  ```
  - In this case, the data in the JSON lines entries must include `instruction`, `context` and `response` fields. If a custom template is provided it must also use `prompt` and `completion` keys to define
  the input and output templates.
  Below is a sample custom template:

  ```json
  {
    "prompt": "question: {question} context: {context}",
    "completion": "{answer}"
  }
  ```
Here, the data in the JSON lines entries must include `question`, `context` and `answer` fields. 
- **Output:** A trained model that can be deployed for inference. 


#### 2.3. Example fine-tuning with Domain-Adaptation dataset format
We provide a subset of SEC filings data of Amazon in domain adaptation dataset format. It is downloaded from publicly available [EDGAR](https://www.sec.gov/edgar/searchedgar/companysearch). Instruction of accessing the data is shown [here](https://www.sec.gov/os/accessing-edgar-data).

License: [Creative Commons Attribution-ShareAlike License (CC BY-SA 4.0)](https://creativecommons.org/licenses/by-sa/4.0/legalcode).

To fine-tune the model on a domain adaptation dataset, you can use the following code.
```python
import boto3
model_id = "meta-textgeneration-llama-2-7b" # modify model_id to the llama model_id you want to use

estimator = JumpStartEstimator(model_id=model_id,  environment={"accept_eula": "true"}, instance_type = "ml.g5.24xlarge")
estimator.set_hyperparameters(instruction_tuned="False", epoch="5")
estimator.fit({"training": f"s3://jumpstart-cache-prod-{boto3.Session().region_name}/training-datasets/sec_amazon"})
```


### 3. Supported Hyper-parameters for fine-tuning

- epoch: The number of passes that the fine-tuning algorithm takes through the training dataset. Must be an integer greater than 1. Default: 5
- learning_rate: The rate at which the model weights are updated after working through each batch of training examples. Must be a positive float greater than 0. Default: 1e-4.
- instruction_tuned: Whether to instruction-train the model or not. Must be 'True' or 'False'. Default: 'False'
- per_device_train_batch_size: The batch size per GPU core/CPU for training. Must be a positive integer. Default: 4.
- per_device_eval_batch_size: The batch size per GPU core/CPU for evaluation. Must be a positive integer. Default: 1
- max_train_samples: For debugging purposes or quicker training, truncate the number of training examples to this value. Value -1 means using all of training samples. Must be a positive integer or -1. Default: -1. 
- max_val_samples: For debugging purposes or quicker training, truncate the number of validation examples to this value. Value -1 means using all of validation samples. Must be a positive integer or -1. Default: -1. 
- max_input_length: Maximum total input sequence length after tokenization. Sequences longer than this will be truncated. If -1, max_input_length is set to the minimum of 1024 and the maximum model length defined by the tokenizer. If set to a positive value, max_input_length is set to the minimum of the provided value and the model_max_length defined by the tokenizer. Must be a positive integer or -1. Default: -1. 
- validation_split_ratio: If validation channel is none, ratio of train-validation split from the train data. Must be between 0 and 1. Default: 0.2. 
- train_data_split_seed: If validation data is not present, this fixes the random splitting of the input training data to training and validation data used by the algorithm. Must be an integer. Default: 0.
- preprocessing_num_workers: The number of processes to use for the preprocessing. If None, main process is used for preprocessing. Default: "None"
- lora_r: Lora R. Must be a positive integer. Default: 8.
- lora_alpha: Lora Alpha. Must be a positive integer. Default: 32
- lora_dropout: Lora Dropout. must be a positive float between 0 and 1. Default: 0.05. 
- int8_quantization: If True, model is loaded with 8 bit precision for training. Default for 7B/13B: False. Default for 70B: True.
- enable_fsdp: If True, training uses Fully Sharded Data Parallelism. Default for 7B/13B: True. Default for 70B: False.

Note 1: int8_quantization is not supported with FSDP. Also, int8_quantization = 'False' and enable_fsdp = 'False' is not supported due to CUDA memory issues for any of the g5 family instances. Thus, we recommend setting exactly one of int8_quantization or enable_fsdp to be 'True'
Note 2: Due to the size of the model, 70B model can not be fine-tuned with enable_fsdp = 'True' for any of the supported instance types.


### 4. Supported Instance types

We have tested our scripts on the following instances types:

| Model | Model ID | All Supported Instances Types for fine-tuning |
| - | - | - |
| Llama 2 7B | meta-textgeneration-llama-2-7b | ml.g5.12xlarge, ml.g5.24xlarge, ml.g5.48xlarge, ml.p3dn.24xlarge |
| Llama 2 13B | meta-textgeneration-llama-2-13b | ml.g5.24xlarge, ml.g5.48xlarge, ml.p3dn.24xlarge |
| Llama 2 70B | meta-textgeneration-llama-2-70b | ml.g5.48xlarge |
| Llama 3 8B | meta-textgeneration-llama-3-8b | ml.g5.12xlarge, ml.g5.24xlarge, ml.g5.48xlarge, ml.p3dn.24xlarge, ml.g4dn.12xlarge |
| Llama 3 70B | meta-textgeneration-llama-3-70b | ml.g5.48xlarge, ml.p4d.24xlarge |
| Llama 3.1 8B | meta-textgeneration-llama-3-1-8b | ml.g5.12xlarge, ml.g5.24xlarge, ml.g5.48xlarge, ml.p3dn.24xlarge, ml.g4dn.12xlarge |
| Llama 3.1 70B | meta-textgeneration-llama-3-1-70b | ml.g5.48xlarge, ml.p4d.24xlarge |
| Llama 3.1 405B FP8 | meta-textgeneration-llama-3-1-405b-fp8 | ml.p5.48xlarge |


Other instance types may also work to fine-tune. Note: When using p3 instances, training will be done with 32 bit precision as bfloat16 is not supported on these instances. Thus, training job would consume double the amount of CUDA memory when training on p3 instances compared to g5 instances.


### 5. Few notes about the fine-tuning method

- Fine-tuning scripts are based on [this repo](https://github.com/facebookresearch/llama-recipes/tree/main). 
- Instruction tuning dataset is first converted into domain adaptation dataset format before fine-tuning. 
- Fine-tuning scripts utilize Fully Sharded Data Parallel (FSDP) as well as Low Rank Adaptation (LoRA) method fine-tuning the models


### 6. Studio Kernel Dead/Creating JumpStart Model from the training Job

Due to the size of the Llama 70B model, training job may take several hours and the studio kernel may die during the training phase. However, during this time, training is still running in SageMaker. If this happens, you can still deploy the endpoint using the training job name with the following code:
```python
from sagemaker.jumpstart.estimator import JumpStartEstimator
training_job_name = <<training_job_name>>

attached_estimator = JumpStartEstimator.attach(training_job_name, model_id)
attached_estimator.logs()
attached_estimator.deploy()
```

How do you find the training job name? Go to Console -> SageMaker -> Training -> Training Jobs -> Identify the training job name and substitute in the following cell. 
